In [ ]:
# Import Required Libraries
from DatabasePackage import excel_to_assets_json

In [2]:
# Generate Asset Database from Excel File
# This cell reads the Asset_database.xlsx file and converts it to a JSON format
# with the structure: {Area: {Asset: {land_price, house_price, rent {...}}}}
# The resulting Asset_database.json file is created in the current directory
excel_to_assets_json("Asset_database.xlsx")

/Users/rupam/Projects/Monopoly app/DatabasePackage/player_manager.py:33: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df = df.replace(r'^\s*$', pd.NA, regex=True).dropna(axis=0, how='all').reset_index(drop=True)
